In [1]:
import pandas as pd 
import numpy as np
import math
import cmath
from sympy import * 
import scipy.io as sio



In [2]:
## This code read the intersection matrix of linear cycles, AS cycles, t3 cycles and build the
## total intersection matrix.

## We use the correction lc-l0, AS-6*l0, t3-12*l0, where l0 is the first linear cycle. The initial 
## of linear cycles are 1002.

In [3]:
def replace_values(list_to_replace, item_to_replace, item_to_replace_with):
    return [item_to_replace_with if item == item_to_replace else item for item in list_to_replace]



#1) Intersection matrix of linear cycles:
ruta=open("Intersection_matrix_lc.txt","r")
IMlc=ruta.read()
IMlc=IMlc.split(",")
IMlc=replace_values(IMlc,'125/6\n','125/6')
IMlc=np.array(IMlc)
IMlc=np.reshape(IMlc, (1002, 1002))

#2) Intersection matrix of AS cycles:
ruta=open("Intersection_matrix_AS.txt","r")
IMAS=ruta.read()
IMAS=IMAS.split(",")
IMAS=replace_values(IMAS,'60\n','60')
IMAS=np.array(IMAS)
IMAS=np.reshape(IMAS, (720, 720))


#3) Intersection matrix of t3 cycles:
ruta=open("Intersection_matrix_t3.txt","r")
IMt3=ruta.read()
IMt3=IMt3.split(",")
IMt3=replace_values(IMt3,'96\n','96')
IMt3=np.array(IMt3)
IMt3=np.reshape(IMt3, (30, 30))


#4) Intersection matrix of linear cycles with t3 cycles:
ruta=open("Intersection_matrix_lc_t3.txt","r")
IMlct3=ruta.read()
IMlct3=IMlct3.split(",")
IMlct3=replace_values(IMlct3,'-1\n','-1')
IMlct3=np.array(IMlct3)
IMlct3=np.reshape(IMlct3, (1002, 30))




In [4]:
## Definition of the total intersection matrix
IM=np.zeros([1751,1751]);

## linear cycles: <lc_i-l0,lc_j-l0>=<lc_i,lc_j>-<lc_i,l0>-<lc_j,l0>+<l0,l0>
for i in range(1,1002):
    for j in range(1,1002):
        IM[i-1,j-1]=eval(IMlc[i,j])-eval(IMlc[i,0])-eval(IMlc[0,j])+eval(IMlc[0,0]);
        
## AS cycles: <AS_i-6*l0,AS_j-6*l0>=<AS_i,AS_j>+36<l0,l0>
for i in range(720):
    for j in range(720):
        IM[i+1001,j+1001]=eval(IMAS[i,j])+36*eval(IMlc[0,0]);
        
## t3 cycles: <t3_i-12*l0,t3_j-12*l0>=<t3_i,t3_j>-12<t3_i,l0>-12<l0,t3_j>+144<l0,l0>
for i in range(30):
    for j in range(30):
        IM[i+1721,j+1721]=eval(IMt3[i,j])-12*eval(IMlct3[0,i])-12*eval(IMlct3[0,j])+144*eval(IMlc[0,0]);
        
## linear cycles and AS cycles: <lc_i-l0,AS_j-6*l0>=-6<lc_i,l0>+6<l0,l0>
for i in range(1,1002):
    for j in range(720):
        IM[i-1,j+1001]=-6*eval(IMlc[i,0])+6*eval(IMlc[0,0]);
        IM[j+1001,i-1]=-6*eval(IMlc[i,0])+6*eval(IMlc[0,0]);

## linear cycles and t3 cycles: <lc_i-l0,t3_j-12*l0>=<lc_i-t3_j>-<l0,t3_j>-12<lc_i,l0>+12<l0,l0>
for i in range(1,1002):
    for j in range(30):
        IM[i-1,j+1721]=eval(IMlct3[i,j])-eval(IMlct3[0,j])-12*eval(IMlc[i,0])+12*eval(IMlc[0,0]);
        IM[j+1721,i-1]=eval(IMlct3[i,j])-eval(IMlct3[0,j])-12*eval(IMlc[i,0])+12*eval(IMlc[0,0]);
        
## AS cycles and t3 cycles: <AS_i-6*l0,t3_j-12*l0>=-6<l0,t3_j>+72<l0,l0>
for i in range(720):
    for j in range(30):
        IM[i+1001,j+1721]=-6*eval(IMlct3[0,j])+72*eval(IMlc[0,0]);
        IM[j+1721,i+1001]=-6*eval(IMlct3[0,j])+72*eval(IMlc[0,0]);

        

In [5]:
## Definition of the total intersection matrix SIN RESTAR L0
IM1=np.zeros([1751,1751]);

## linear cycles:
for i in range(1,1002):
    for j in range(1,1002):
        IM1[i-1,j-1]=eval(IMlc[i,j]);
        
## AS cycles: 
for i in range(720):
    for j in range(720):
        IM1[i+1001,j+1001]=eval(IMAS[i,j]);
        
## t3 cycles:
for i in range(30):
    for j in range(30):
        IM1[i+1721,j+1721]=eval(IMt3[i,j]);
        

## linear cycles and t3 cycles:
for i in range(1,1002):
    for j in range(30):
        IM1[i-1,j+1721]=eval(IMlct3[i,j]);
        IM1[j+1721,i-1]=eval(IMlct3[i,j]);
        

In [6]:
np.linalg.matrix_rank(IM)

1750

In [7]:
np.linalg.matrix_rank(IM1)

1751

In [8]:
sio.savemat('Intersection_matrix', {"IM": IM })

In [9]:
sio.savemat('Intersection_matrix_sin_correccion_de_restar_l0', {"IM1": IM1 })